In [3]:
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest

sc = SparkContext(appName="TitanicPredictor")


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=TitanicPredictor, master=local[*]) created by __init__ at <ipython-input-1-a8651f1b581d>:5 

In [4]:

train = 'file:/root/notebooks/Titanic/train.csv'
train_data = sc.textFile(train)

test = 'file:/root/notebooks/Titanic/test.csv'
test_data = sc.textFile(test)


print "Train data size is {}".format(train_data.count())
print "Test data size is {}".format(test_data.count())

Train data size is 892
Test data size is 419


In [5]:
#skip header
header = train_data.first() #extract header
train_data = train_data.filter(lambda x:x !=header) #filter out header

test_header = test_data.first()
test_data = test_data.filter(lambda x:x !=test_header) #filter out header

print header
print test_header

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [21]:
# Import SQLContext and data types
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *

# sc is an existing SparkContext.
sqlContext = SQLContext(sc)

#Split the csv file by comma and convert each line to a tuple.
parts = train_data.map(lambda l: l.split(",", -1))
people = parts.map(lambda p: Row(PassengerId=(p[0]), Survived=(p[1]), 
                                 Pclass=p[2], Name=p[3]+','+p[4], Sex=p[5], Age=((p[6])), 
                                 SibSp=(p[7]), Parch=(p[8]), Ticket=p[9], 
                                 Fare=p[10], Cabin=p[11], Embarked=p[12]))

# Apply the schema to the RDD.
schemaPeople = sqlContext.createDataFrame(people)

# Register the DataFrame as a table.
schemaPeople.registerTempTable("titanic")
schemaPeople.printSchema()




root
 |-- Age: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Ticket: string (nullable = true)



In [20]:
results = sqlContext.sql("SELECT Name, Age FROM titanic")
for name in results.collect():
  print(name)

Row(Name=u'"Braund, Mr. Owen Harris"', Age=u'22')
Row(Name=u'"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"', Age=u'38')
Row(Name=u'"Heikkinen, Miss. Laina"', Age=u'26')
Row(Name=u'"Futrelle, Mrs. Jacques Heath (Lily May Peel)"', Age=u'35')
Row(Name=u'"Allen, Mr. William Henry"', Age=u'35')
Row(Name=u'"Moran, Mr. James"', Age=u'')
Row(Name=u'"McCarthy, Mr. Timothy J"', Age=u'54')
Row(Name=u'"Palsson, Master. Gosta Leonard"', Age=u'2')
Row(Name=u'"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"', Age=u'27')
Row(Name=u'"Nasser, Mrs. Nicholas (Adele Achem)"', Age=u'14')
Row(Name=u'"Sandstrom, Miss. Marguerite Rut"', Age=u'4')
Row(Name=u'"Bonnell, Miss. Elizabeth"', Age=u'58')
Row(Name=u'"Saundercock, Mr. William Henry"', Age=u'20')
Row(Name=u'"Andersson, Mr. Anders Johan"', Age=u'39')
Row(Name=u'"Vestrom, Miss. Hulda Amanda Adolfina"', Age=u'14')
Row(Name=u'"Hewlett, Mrs. (Mary D Kingcome) "', Age=u'55')
Row(Name=u'"Rice, Master. Eugene"', Age=u'2')
Row(Name=u'"Williams, Mr. Cha

In [11]:
# Convert Age and Fare to float
df = schemaPeople.withColumn("Agetmp", schemaPeople.Age.cast('double')).drop("Age").withColumnRenamed("Agetmp","Age").withColumn("Faretmp", schemaPeople.Fare.cast('double')).drop("Fare").withColumnRenamed("Faretmp","Fare")
df.printSchema()



root
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Fare: double (nullable = true)



In [18]:
results = sqlContext.sql("SELECT Name, Age FROM titanic")
for name in results.collect():
  print(name)

Row(Name=u'"Braund, Mr. Owen Harris"', Age=u'22')
Row(Name=u'"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"', Age=u'38')
Row(Name=u'"Heikkinen, Miss. Laina"', Age=u'26')
Row(Name=u'"Futrelle, Mrs. Jacques Heath (Lily May Peel)"', Age=u'35')
Row(Name=u'"Allen, Mr. William Henry"', Age=u'35')
Row(Name=u'"Moran, Mr. James"', Age=u'')
Row(Name=u'"McCarthy, Mr. Timothy J"', Age=u'54')
Row(Name=u'"Palsson, Master. Gosta Leonard"', Age=u'2')
Row(Name=u'"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"', Age=u'27')
Row(Name=u'"Nasser, Mrs. Nicholas (Adele Achem)"', Age=u'14')
Row(Name=u'"Sandstrom, Miss. Marguerite Rut"', Age=u'4')
Row(Name=u'"Bonnell, Miss. Elizabeth"', Age=u'58')
Row(Name=u'"Saundercock, Mr. William Henry"', Age=u'20')
Row(Name=u'"Andersson, Mr. Anders Johan"', Age=u'39')
Row(Name=u'"Vestrom, Miss. Hulda Amanda Adolfina"', Age=u'14')
Row(Name=u'"Hewlett, Mrs. (Mary D Kingcome) "', Age=u'55')
Row(Name=u'"Rice, Master. Eugene"', Age=u'2')
Row(Name=u'"Williams, Mr. Cha

In [22]:
# Convert Age and Fare to float for data
df = schemaPeople.withColumn("Agetmp", schemaPeople.Age.cast('double'))
                .drop("Age")
                .withColumnRenamed("Agetmp","Age")
                .withColumn("Faretmp", schemaPeople.Fare.cast('double'))
                .drop("Fare")
                .withColumnRenamed("Faretmp","Fare")
df.printSchema()

root
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Fare: double (nullable = true)



In [8]:
# Create Dataframe for Test data
test_parts = test_data.map(lambda l: l.split(",", -1))
testPeople = test_parts.map(lambda p: Row(PassengerId=(p[0]),  
                                 Pclass=p[1], Name=p[2]+','+p[3], Sex=p[4], Age=((p[5])), 
                                 SibSp=(p[6]), Parch=(p[7]), Ticket=p[8], 
                                 Fare=p[9], Cabin=p[10], Embarked=p[11]))

# Apply the schema to the RDD.
test_schemaPeople = sqlContext.createDataFrame(testPeople)

# Register the DataFrame as a table.
test_schemaPeople.registerTempTable("titanic_test")

# Convert Age and Fare to float for test data
test_df = test_schemaPeople.withColumn("Agetmp", test_schemaPeople.Age.cast('double'))
                        .drop("Age")
                        .withColumnRenamed("Agetmp","Age")
                        .withColumn("Faretmp", test_schemaPeople.Fare.cast('double'))
                        .drop("Fare")
                        .withColumnRenamed("Faretmp","Fare")
test_df.printSchema()

root
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Fare: double (nullable = true)



In [24]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Bucketizer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler



# StringIndexer for Pclass and Sex
pclassIndexer = StringIndexer(inputCol="Pclass", outputCol="PclassIndex")
sexIndexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")

#Bucketizer for Age and Fare
ageSplits = [0.0, 5.0, 12.0, 18.0, 30.0, 60.0, 120.0]
fareSplits = [0.0, 5.0, 10.0, 30.0, 50.0, 75.0, 100.0, float("inf")]

ageBucketizer = Bucketizer(splits=ageSplits, inputCol="Age", outputCol="bucketedAge")
fareBucketizer = Bucketizer(splits=fareSplits, inputCol="Fare", outputCol="bucketedFare")


#Vector Assembler for features
featureAssembler = VectorAssembler(
   # inputCols=["PclassIndex", "SexIndex", "bucketedAge"],
    inputCols=["PclassIndex", "SexIndex", "bucketedAge", "bucketedFare"],
    outputCol="features")

# Label Indexer using StringIndexer
labelIndexer = StringIndexer(inputCol="Survived", outputCol="indexedLabel")



In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="features")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[pclassIndexer, sexIndexer, labelIndexer, 
                            ageBucketizer, fareBucketizer, featureAssembler, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(df)

treeModel = model.stages[6]
print treeModel # summary only



DecisionTreeClassifier_42a090ddd04b15851a1b


In [27]:
import numpy
from numpy import array

# Make predictions.
predictions = model.transform(test_df).select("PassengerId", "prediction")
predictionStrings = predictions.withColumn("Survived", predictions.prediction.cast('int'))
                    .drop("prediction")
predictions.printSchema()

# Select example rows to display.
output= predictionStrings.rdd
header = sc.parallelize(array(["PassengerId,Survived"]))
lines = output.map(lambda l: str(l[0]) + ',' + str(l[1]))
header.union(lines).saveAsTextFile('file:/root/notebooks/Titanic/predicted_output.csv')

#Kaggle score = 0.7521 on first attempt

root
 |-- PassengerId: string (nullable = true)
 |-- prediction: double (nullable = true)



In [18]:
from pyspark.sql import SQLContext, Row
from pyspark.ml import Pipeline
from pyspark.ml import Transformer
from pyspark.ml.feature import Tokenizer, HashingTF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param

sqlContext = SQLContext(sc)

def getLabelAndFeats(line):
    """
    Parse a line of text into an MLlib LabeledPoint object.
    """
    values = line.split(',',2)
    return Row(label=float(values[1]), text=values[2])

training = train_data.map(getLabelAndFeats).toDF()
training.printSchema()
print training.take(2)



root
 |-- label: double (nullable = true)
 |-- text: string (nullable = true)

[Row(label=0.0, text=u'3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S'), Row(label=1.0, text=u'1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C')]
